In [2]:
import torch
from tqdm import tqdm
import tensorflow as tf
import torch.nn as nn
from torchvision import transforms, datasets
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data.dataset import Dataset
from torch import Tensor
import utils
import torch.nn.functional as F
import string
alphabet=' '+string.ascii_letters+string.punctuation+string.digits
converter = utils.strLabelConverter(alphabet,ignore_case=False)

In [3]:
from torchvision.models.resnet import resnet50
model=resnet50(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
newmodel = nn.Sequential(*(list(model.children())[:-5]))
block2=nn.Sequential(nn.Conv2d(256,128,kernel_size=(3,7),stride=(1,1),bias=False),
              nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              nn.Conv2d(128, 128, kernel_size=(3, 7), stride=(1, 1), bias=False),
              nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              nn.Conv2d(128, 512, kernel_size=(3, 7), stride=(1, 1), bias=False),
              nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
              nn.ReLU(inplace=True),
              nn.Conv2d(512, 512, kernel_size=(3, 7), stride=(1,1), bias=False),
              nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             )
block3=nn.Sequential(nn.Conv2d(512, 128, kernel_size=(3, 3), stride=(1, 1), bias=False),
                     nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                     nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=False),
                     nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                     nn.Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), bias=False),
                     nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                     nn.ReLU(inplace=True)
                    )
block4=nn.Sequential(nn.Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), bias=False),
                     nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                     nn.Conv2d(256, 256, kernel_size=(3, 3), bias=False),
                     nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                     nn.Conv2d(256, 1024, kernel_size=(3, 3), stride=(1, 1), bias=False),
                     nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                     nn.ReLU(inplace=True),
                     nn.Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), bias=False),
                     nn.BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                     
)

In [4]:
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN,self).__init__()
        self.cnn=newmodel
        self.block2=block2
        self.block3=block3
        self.block4=block4
        self.rnn_layer=nn.RNN(1024,hidden_size=256,num_layers=2,bidirectional=True)
        self.num_classes = len(alphabet)
        self.linear = nn.Linear(256 * 2,self.num_classes+1)
        self.softmax = nn.LogSoftmax(dim=2)
        self.abc=alphabet
    def forward(self,X):
        hidden = self.init_hidden(X.size(0), next(self.parameters()).is_cuda)
        output=self.cnn(X)
        #print (output.size())
        output=self.block2(output)
        #print (output.size())
        output=self.block3(output)
        #print (output.size())
        output=self.block4(output)
        #print (output.size())
        conv = output.squeeze(2)
        conv = conv.permute(2, 0, 1)
        #print (conv.size())
        seq,hidden=self.rnn_layer(conv,hidden)
        
        seq = self.linear(seq)
        seq = self.softmax(seq)
        #if not self.training:
        #    seq = self.softmax(seq)
        #    if decode:
        #        seq = self.decode(seq)
        return seq
    def init_hidden(self, batch_size, gpu=True):
        h0 = Variable(torch.zeros(2 * 2,batch_size,256))
        if gpu:
            h0 = h0.cuda()
        return h0
    

    def pred_to_string(self, pred):
        seq = []
        for i in range(pred.shape[0]):
            label = np.argmax(pred[i])
            seq.append(label - 1)
        out = []
        for i in range(len(seq)):
            if len(out) == 0:
                if seq[i] != -1:
                    out.append(seq[i])
            else:
                if seq[i] != -1 and seq[i] != seq[i - 1]:
                    out.append(seq[i])
        out = ''.join(self.abc[i] for i in out)
        return out

    def decode(self, pred):
        pred = pred.permute(1, 0, 2).cpu().data.numpy()
        seq = []
        for i in range(pred.shape[0]):
            seq.append(self.pred_to_string(pred[i]))
        return seq

In [5]:
from PIL import Image
import pandas as pd
import string
from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence
import numpy as np
class CustomDatasetFromImages(Dataset):
    def __init__(self, csv_path,transform=None):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        self.data_info = pd.read_csv(csv_path, header=None,sep='|')
        self.data_info=self.data_info#[:32]
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        self.label = self.data_info.iloc[:, 1].apply(lambda x:torch.cuda.IntTensor(self.text_to_labels(x)))
        self.seqLength=[]
        for j in self.label:
            self.seqLength.append(len(j))
        self.transforms = transform
        self.label_arr = pad_sequence(self.label,batch_first=True)
        #print (self.label_arr)
        self.data_len = len(self.data_info.index)
        
    def __getitem__(self, index):
        
        single_image_name = self.image_arr[index]
        img_as_img = Image.open(single_image_name)
        img_as_img = img_as_img.convert('L')
        img_as_tensor = self.transforms(img_as_img)
        #print (img_as_tensor.size())
        single_image_label = self.label_arr[index]
        #print (single_image_label)
        batch = {"img": img_as_tensor.cuda(), "seq": single_image_label,'seq_len':self.seqLength[index]}
        return batch

    def __len__(self):
        return self.data_len
    def text_to_labels(self,text):
        ret = []
        for char in text:
            ret.append(alphabet.find(char))
        return ret

In [6]:
transformations = transforms.Compose([transforms.Resize((90,540)),transforms.ToTensor()])
data_set=CustomDatasetFromImages('htr_data.csv',transform=transformations)
mn_dataset_loader = torch.utils.data.DataLoader(dataset=data_set,batch_size=63,shuffle=False)
def init_weights(m):
    if type(m) == nn.Linear or type(m)==nn.Conv2d:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [7]:
train_size = int(0.9 * len(mn_dataset_loader))
test_size = len(mn_dataset_loader) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(mn_dataset_loader, [train_size, test_size])

In [8]:
len(train_dataset)*64

10368

In [ ]:
cmodel = CRNN().cuda()
#print (output.size())cmodel.apply(init_weights)
criterion = nn.CTCLoss(blank=0)
optimizer = torch.optim.RMSprop(cmodel.parameters(), lr=0.0005)
iterator = tqdm(mn_dataset_loader)
#print (iterator)
epoch=0
while True:
    running_loss = 0.0
    if epoch==500:
        break
    step=0
    for item in mn_dataset_loader:
        step+=1
        images = Variable(item['img'])
        labels = Variable(item['seq'])
        #print (images.size())
        #print (labels.size())
        optimizer.zero_grad()
        outputs = cmodel(images)
        #if step%10==0:
        #    p=outputs[0]

        #    decodes, _ = tf.nn.ctc_beam_search_decoder(inputs=p.cpu().detach().numpy(),sequence_length=99*np.ones(1), merge_repeated=True)
        #    with tf.Session(config = tf.ConfigProto(device_count = {'GPU': 0})) as sess:
        #        t_ = sess.run(decodes)[0].values
        #        char_list = []
        #        for i in range(len(sess.run(decodes)[0].values)):
        #                if t_[i] != 0 and (not (i > 0 and t_[i - 1] == t_[i])):
        #                    char_list.append(alphabet[t_[i] - 1])
        #        sim_pred = ''.join(char_list)
        #        print (sim_pred)
        pred_lens = Variable(Tensor([outputs.size(0)] * 63).int())
        label_lens = Variable(item['seq_len'].int())
        loss = criterion(outputs,labels,pred_lens,label_lens)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
            
    epoch+=1
    print(epoch,running_loss/step)
    #n, preds = outputs.max(2)
    #print (n)
    #print ('done')
    #print (preds)
    #preds=
    #preds = preds.squeeze(2)
    #print (preds)
    #preds = preds.contiguous().view(-1)
    #preds_size = Variable(torch.Inteduction='mean'Tensor([preds.size(0)]))
    #raw_pred = converter.decode(preds.data, preds_size.data, raw=False)
    #print (len(raw_pred))
    #sim_pred = model.decode(outputs)
    #print (raw_pred)
    #print ('\n')
    #print (raw_pred)

  0%|          | 0/180 [00:00<?, ?it/s]

1 2.7844668640030754
2 2.506257116794586
3 2.418787129720052
4 2.38413626882765
5 2.316758852534824
6 2.3656846920649213
7 2.3521887010998195
8 2.298591024345822
9 2.2927293512556286
10 2.325178152985043
11 2.2952105601628623
12 2.2836824019749957
13 2.268980529573229
14 2.2752033405833774
15 2.2560946663220722
16 2.264959670437707
17 2.252026723490821
18 2.2575770378112794
19 2.2488665368821885
20 2.2470244619581434
21 2.2539031128088634
22 2.2475311199824017
23 2.258820797337426
24 2.2546579082806906
25 2.2529238051838343
26 2.243513778845469
27 2.242825946542952
28 2.234832379553053
29 2.23793809082773
30 2.2277292013168335
31 2.2357754680845474
32 2.2457206500901115
33 2.23326736357477
34 2.228330859210756
35 2.2260054296917384
36 2.2228798296716477
37 2.223023058970769
38 2.2178168813387553
39 2.217941257688734
40 2.221052204238044
